In [1]:
import numpy as np
#from load_data import load_mnist
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from sklearn.metrics import pairwise_distances
import seaborn as sns

#palette=sns.color_palette("cubehelix", 10)
palette=sns.color_palette("magma", 10)
#palette=sns.color_palette("viridis", 10)

In [2]:
#Load the data
X, y = load_digits(return_X_y=True) #X a np array of size (1797, 64) and y the corresponding label
print(X.shape)
print(len(y))

(1797, 64)
1797


### Function for easy plots

In [3]:
#Plots

def plot_tsne(xy, hue='hue', legend = "full", palette = palette):
    sns.set(rc={'figure.figsize':(10,10)})
    #palette=sns.color_palette("cubehelix", 10)
    palette=sns.color_palette("magma", 10)
    #palette=sns.color_palette("viridis", 10)
    fig = sns.scatterplot(xy[:,0], xy[:,1],
                hue = hue,
                legend = "full",
                palette = palette) # don't use edges
    plt.show()
    
#plot_tsne(X_embedded, hue = y)

## Helper functions

In [4]:
def pair_distances(X):
    D_squared = np.sum((X[None,:] - X[:, None])**2, -1)
    return -D_squared

In [ ]:
distances = pair_distances(X)

In [ ]:
distances = pair_distances(X)

In [5]:
#This is for equation 1 and 4 
def quotient(distances):
    # Subtract max for numerical stability
    e_x = np.exp(distances - np.max(distances, axis=1).reshape([-1, 1]))

    # Diagonal 0.
    np.fill_diagonal(e_x, 0.)

    # Add a tiny constant for stability of log we take later
    e_x = e_x + 1e-6  # numerical stability

    return e_x / e_x.sum(axis=1).reshape([-1, 1])

In [45]:
#None for q_ij and sigmas activated for p_ij
def compute_proba_matrix(distances,sigmas=None):
    if sigmas.all()==None: 
        return quotient(distances)
    else:
        return quotient(distances / (2*np.square(sigmas.reshape((-1, 1))))) #WARNING: HERE WE ARE USING THE STANDARD DEVIATION (?) INSTEAD OF THE VARIANCE 

In [ ]:
compute_proba_matrix(distances,sigmas=None)

In [8]:
def solve_crowd(P):
    N = P.shape[0]
    return (P+P.T)/2*N

In [9]:
#Compute the low dimesional map qij using ec 4
def Q_table(Y):
    distances_low_dim=pair_distances(Y)
    nom = np.linalg.inv(1.-distances_low_dim)
    # Fill diagonal with zeroes so q_ii = 0
    np.fill_diagonal(exp_distances, 0.) # WARNING: NUMERICAL STABILITY
    deno = np.linalg.inv(np.sum(1.-exp_distances))
    return nom/deno

def q_joint(Y):
    """Given low-dimensional representations Y, compute
    matrix of joint probabilities with entries q_ij."""
    # Get the distances from every point to every other
    distances = neg_squared_euc_dists(Y)
    # Take the elementwise exponent
    exp_distances = np.exp(distances)
    # Fill diagonal with zeroes so q_ii = 0
    np.fill_diagonal(exp_distances, 0.)
    # Divide by the sum of the entire exponentiated matrix
    return exp_distances / np.sum(exp_distances), None

In [34]:
def P_table(X,our_perplexity):
    distances_high_dim = pair_distances(X)
    print(f"Distances {distances_high_dim}")
    
    sigmas=find_sigmas(distances_high_dim,our_perplexity)
    print(f"Sigmas {sigmas}")
    P=compute_proba_matrix(distances_high_dim,sigmas)
    return solve_crowd(P)

In [11]:
#def compute_gradient(P,Q,Y):
    
#    pq_diff = P - Q
#    pq_expanded = np.expand_dims(pq_diff, 2)
#    y_diffs = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
#    
#    distances_low_dim=pair_distances(Y)
#    aux = np.linalg.inv(np.exp(1.-distances))
#    distances_low_expanded = np.expand_dims(aux, 2)
#    
#    y_diffs_wt = y_diffs * distances_low_expanded#
#
#    # Multiply then sum over j's
#    grad = 4. * (pq_expanded * y_diffs_wt).sum(1)
#    
#    return grad

def gradient(p, q, y):
    """Computes the gradient"""
    N = p.shape[0]
    m = y.shape[1]
    grad = np.zeros((N,m))
    for i in range(N):
        for j in range(N):
            grad[i,:] += 4*(p[i,j] - q[i,j])*(y[i] - y[j])*(1 + np.linalg.norm(y[i] - y[j])**2)**(-1)
    return grad

In [41]:
def t_sne(X,classes,our_perplexity, num_iter, nu, alpha,P):
    
    N = X.shape[0]
    print(f"Number of points: {N}")
    #P = P_table(X,our_perplexity)
    #print(f"P table {P}")
    np.random.seed(10)
    Y = np.random.normal(loc=0.0, scale=1e-4, size=(N,2))
    Y_m2 = Y.copy()
    Y_m1 = Y.copy()
    for i in range(num_iter):
        print(i)
        Q= Q_table(Y) #compute low dimensional affinities
        print(f"Q table {Q}")
        grad = compute_gradient(P,Q,Y) # compute gradient
        print(f"Print gradient: {grad}")
        Y = Y + nu*grad + alpha*(Y_m1-Y_m2)#set y_i's
        Y_m2 = Y_m1.copy()
        Y_m1 = Y.copy()
    
    return Y #low dimensional representation

### Section to compute the sigmas.

This is use to compute p_j|i given that we need one sigma per point in the space

In [13]:
#First a binary search 

#To locate the index of the suitable sigma in array and later on
def binary_search(array,target,tol=1e-10,max_iter=1000,lower=1e-20,upper=1000.):
    for i in range(max_iter):
        guess = ((lower + upper) / 2.) + lower
        val = array(guess)
        if val > target:
            upper = guess
        else:
            lower = guess
        if np.abs(val - target) <= tol:
            break
    return guess

In [14]:
#The remaining parameter to be selected is the variance si of the Gaussian that is centered over
#each high-dimensional datapoint, xi. It is not likely that there is a single value of si that is optimal
#for all datapoints in the data set because the density of the data is likely to vary. In dense regions,
#a smaller value of si is usually more appropriate than in sparser regions. Any particular value of
#si induces a probability distribution, Pi, over all of the other datapoints. This distribution has an
#entropy which increases as si increases. SNE performs a binary search for the value of si that
#produces a Pi with a fixed perplexity that is specified by the user

def calc_perplex(distances,sigmas):
    proba_matrix=compute_proba_matrix(distances,sigmas=sigmas)
    logar=np.log2(proba_matrix)
    entropy = -np.sum(proba_matrix*logar,1)
    perplexity = 2**entropy
    return perplexity

In [15]:
#SNE performs a binary searh for the value of sigma_i that ptoduces a P_i with a
#fixed perplexity that is specified by the user
def find_sigmas(distances,our_perplexity):
    sigmas=np.zeros(distances.shape[0])
    # For each row of the matrix (each point in our dataset)
    for i in range(distances.shape[0]):
        # Make fn that returns perplexity of this row given sigma
        eval_sigma_fn = lambda sigma: \
            calc_perplex(distances[i:i+1, :], np.array(sigma))
        # Binary search over sigmas to achieve target perplexity
        correct_sigma = binary_search(eval_sigma_fn, our_perplexity,tol=1e-10,max_iter=1000,lower=1e-20,upper=1000.)
        # Save the correct sigma 
        sigmas[i]=correct_sigma
        print(sigmas)
    return np.array(sigmas) #Do we need the np.array??? sigmas is already an array, isn't it?

## Implement t-sne

In [46]:
# Set global parameters
#NUM_POINTS = 100            # Number of samples from MNIST
#CLASSES_TO_USE = [0, 1, 8]  # MNIST classes to use
PERPLEXITY = 30                   # Random seed
MOMENTUM = 0.9              #alpha  
LEARNING_RATE = 10.         # nu
NUM_ITERS = 1             # Num iterations to train for  


# numpy RandomState for reproducibility
rng = np.random.RandomState(1)

# Obtain matrix of joint probabilities p_ij
P = P_table(X, PERPLEXITY)
print(f"P table {P}")

# Fit SNE or t-SNE
X_embeded = t_sne(X=X,classes=y,our_perplexity=PERPLEXITY, num_iter=NUM_ITERS, 
          nu=LEARNING_RATE, alpha=MOMENTUM, P=P)

import winsound
duration = 700  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


Distances [[   -0. -3547. -2930. ... -2538. -1374. -2212.]
 [-3547.    -0. -1733. ... -1489. -2359. -2533.]
 [-2930. -1733.    -0. ... -1470. -2578. -1932.]
 ...
 [-2538. -1489. -1470. ...    -0. -1950.  -834.]
 [-1374. -2359. -2578. ... -1950.    -0. -1554.]
 [-2212. -2533. -1932. ...  -834. -1554.    -0.]]
Sigmas [11.71875 23.4375  23.4375  ... 23.4375  23.4375  23.4375 ]
P table [[5.90334911e-05 1.36838233e-01 2.92356657e-01 ... 3.24885255e-01
  1.05446811e+00 4.30068230e-01]
 [1.36838233e-01 6.90314510e-06 1.57978788e+00 ... 1.72663949e+00
  7.52428291e-01 6.58199223e-01]
 [2.92356657e-01 1.57978788e+00 8.39678158e-06 ... 1.95269810e+00
  6.87915001e-01 1.26612669e+00]
 ...
 [3.24885255e-01 1.72663949e+00 1.95269810e+00 ... 6.48863668e-06
  1.05667231e+00 2.99311568e+00]
 [1.05446811e+00 7.52428291e-01 6.87915001e-01 ... 1.05667231e+00
  5.97978594e-06 1.49235513e+00]
 [4.30068230e-01 6.58199223e-01 1.26612669e+00 ... 2.99311568e+00
  1.49235513e+00 6.30036112e-06]]
Number of point

NameError: name 'distances' is not defined

In [38]:
import winsound
duration = 500  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
def beep():
    print "\a"
 
beep()